In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
data = pd.read_csv('./dataset.csv')

remove = ["Subject", "Activity"]
features = [column for column in list(data.columns) if column not in remove]
data['Gender'].replace(' f', 0, inplace=True)
data['Gender'].replace(' m', 1, inplace=True)
data

,chest_ACC_x,chest_ACC_y,chest_ACC_z,Resp,ECG,wrist_ACC_x,wrist_ACC_y,wrist_ACC_z,wrist_BVP,wrist_TEMP,WEIGHT,Gender,AGE,HEIGHT,SKIN,SPORT,Rpeaks,Activity,Label,Subject
0,0.8564,-0.0678,-0.3656,4.441833,0.015610,-0.765625,-0.078125,0.671875,7.28,32.13,78.0,1,34,182.0,3,6,0,0,49.611369,S1
1,0.8514,-0.0654,-0.3688,4.876709,-0.015747,-0.765625,-0.078125,0.671875,-11.36,32.16,78.0,1,34,182.0,3,6,1,0,49.611369,S1
2,0.8514,-0.0662,-0.3708,3.340149,-0.008743,-0.765625,-0.078125,0.671875,-51.27,32.16,78.0,1,34,182.0,3,6,0,0,49.611369,S1
3,0.8594,-0.0632,-0.3640,0.740051,-0.339523,-0.750000,-0.078125,0.671875,5.65,32.16,78.0,1,34,182.0,3,6,0,0,49.611369,S1
4,0.8510,-0.0674,-0.3694,-1.475525,-0.089905,-0.765625,-0.078125,0.671875,34.34,32.16,78.0,1,34,182.0,3,6,0,0,49.611369,S1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517951,0.8878,0.0122,-0.2330,2.789307,-0.281845,-0.468750,0.859375,0.093750,1.68,33.99,79.0,1,28,183.0,2,5,0,0,79.664230,S15
517952,0.8962,-0.0184,-0.2280,0.563049,0.075851,-0.609375,0.890625,0.265625,-16.38,33.99,79.0,1,28,183.0,2,5,1,0,79.664230,S15
517953,0.9010,0.0006,-0.2102,-1.075745,0.017166,-0.468750,0.531250,-0.562500,13.43,33.99,79.0,1,28,183.0,2,5,0,0,79.664230,S15
517954,0.8906,-0.0124,-0.2238,-2.117920,-0.304184,-0.375000,0.406250,-0.843750,-5.89,33.99,79.0,1,28,183.0,2,5,0,0,79.664230,S15


In [3]:
data.drop(columns = ['WEIGHT','Gender', 'AGE', 'HEIGHT', 'SKIN', 'SPORT', 'Activity','Subject' ], inplace = True)

In [4]:
y = data['Label']
X = data.drop(columns = ['Label'],inplace = False)
X = np.expand_dims(X, axis = 1)
y = np.expand_dims(y, axis = 1)

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train,  y_val = train_test_split(X, y, test_size=0.4, random_state=42)
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

(310773, 1, 11) (310773, 1) (207183, 1, 11) (207183, 1)


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Flatten, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D
from tensorflow.keras.optimizers import Adam, schedules, SGD
DENSE_UNITS = 50
LSTM_UNITS = 50
ACTIVATION_FUNC = 'elu'
FINAL_ACTIVATION = 'elu'
LEARNING_RATE = 0.0001
N_STEPS = int(len(X_train))
N_FEATURES = 11
LOSS_FUNC = 'mean_absolute_error'
OPTIMIZER = Adam(learning_rate=LEARNING_RATE)

In [7]:
#LSTM set
model = Sequential()
model.add(
    LSTM(LSTM_UNITS, activation=ACTIVATION_FUNC, 
         input_shape=(N_STEPS, N_FEATURES) 
))
model.add(Flatten())
model.add(Dense(DENSE_UNITS, activation=ACTIVATION_FUNC))
model.add(Dense(1, activation=FINAL_ACTIVATION))
model.compile(
    optimizer=OPTIMIZER,
    loss=LOSS_FUNC,
    #metrics=[tf.keras.metrics.AUC(), METRIC]
)

In [8]:
model.fit(
    X_train, y_train, epochs=100,
    verbose=1, 
    batch_size=100,
    validation_data=(X_val, y_val),
)

Epoch 1/100


ValueError: in user code:

    File "c:\users\conler\appdata\local\programs\python\python39\lib\site-packages\keras\engine\training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "c:\users\conler\appdata\local\programs\python\python39\lib\site-packages\keras\engine\training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\users\conler\appdata\local\programs\python\python39\lib\site-packages\keras\engine\training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "c:\users\conler\appdata\local\programs\python\python39\lib\site-packages\keras\engine\training.py", line 808, in train_step
        y_pred = self(x, training=True)
    File "c:\users\conler\appdata\local\programs\python\python39\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\users\conler\a